# Generalized Linear Models and Stochastic Gradient Descent in D

The [Mir GLAS](https://github.com/libmir/mir-glas) library has shown that D language is capable of high performance calculations to rival those written in C and C++. Mathematical analysis libraries native to D are however still in short supply. In this blog we roll our own Gamma Generalized Linear Models (GLM) written in optimization style using the Newton-Raphson method and we carry out linear regression using Stochastic Gradient Descent (SGD).

## Mathematical preliminaries

The main aim of a regression algorithm is to find a set of coefficients ($\beta$) that maximize the likelihood of a target variable $(y)$ given a set of explanatory variables $(x)$. The algorithm makes assumptions regarding the distribution of the target variable, and the independence of observations of the explanatory variables.

### Likelihood functions

The likelihood function represents assumption of the distribution of the target variable. The likelhood function for the Gamma distribution is given by:

$$
L(x) = \frac{1}{\Gamma(x)\theta^{k}} x^{k-1} e^{-\frac{x}{\theta}}
$$

The mean of the distribution is given by:

$$
\mu = k \theta
$$

The variance of the distribution is given by:

$$
\sigma^2 = k \theta^2
$$

The GLM dispersion in Gamma is given by:

$$
\hat{\phi} = \frac{X^2}{n - p}
$$

where $n$ is the number of observations and $p$ is the number of parameters and $X^2$ is given by:

$$
X^2 = \sum{\frac{(y_i - \mu_i)^2}{\mu_i^2}}
$$

The parameter $k$ is given by:

$$
k = \frac{1}{\phi}
$$

and the Normal distribution likelihood function is given by:

$$
L(x) = \frac{1}{2\pi \sigma^2}e^{-\frac{(y - \mu)^2}{2\sigma}}
$$

### Gradients and Curvatures

In practice however, we actually maximise the log-likelihood $(l)$ - that is the log of the likelihood function taken over the whole data set. The gradient function for the Gamma log-likelihood (we are assuming a log link $\mu = e^{x\beta}$) is given by

$$
\frac{\partial l}{\partial \beta_j} = -k x_j + k y x_j e^{-x\beta}
$$

and its curvature is given by

$$
\frac{\partial^2 l}{\partial \beta_j \beta_l} = -y x_j x_l e^{-x\beta}
$$

The gradient function for the Normal log-likelihood is given by:

$$
\frac{\partial l}{\partial \beta_j} = x_j \frac{(y - x\beta)}{\sigma^2}
$$

we shall only by doing gradient descent on linear regression but the curvature for the Normal log-likelihood is given below for completeness:

$$
\frac{\partial^2 l}{\partial \beta_j \beta_l} = -\frac{x_l x_j}{\sigma^2}
$$

where $\sigma^2 = \hat{\phi}$

## Regression as Optimization

Regression is thus an optimization algorithm that maximizes the log-likelihood function for a set of $\beta$ coefficients. It can thus be solved by numerical optimization.

### Newton-Raphson Algorithm


The Newton-Raphson method is an iterative optimization algorithm given by:

$$
\beta_{n+1} = \beta_n - H^{-1}g
$$

where $g$ is the gradient vector and $H$ is the curvature matrix. Stopping criterion is 

$$
||\beta_{new} - \beta_{old}|| < \epsilon_1 \quad \cap \quad |l_{new} - l_{old}| < \epsilon_2
$$

### Gradient Descent

The Gradient Descent algorithm is given by:

$$
\beta_{n + 1} = \beta_n + \eta g
$$

where $\eta$ is the learning rate.

## GLM and SGD in D

The scripts for DLM and SGD in D is given in the [D script](https://github.com/dataPulverizer/glm-stochastic-gradient-descent-d/blob/master/code/script.d) file.

### Data I/O

Below is the D function `read_yX` created for reading numeric data from a csv file and returning a `tuple` containing the `x` feature or design matrix and `y` the target variable vector.

<b><code>
auto read_yX(char sep = ',')(string name)
{
	auto file = File(name, "r");
	// Reads data into a double[][] array
	auto data = file.byLine(KeepTerminator.no)
	                .map!(a => a.split(sep)
	                	.map!(b => to!double(b))
	                	.array())
	                .array();
	auto y = data.map!(a => a[0]).array();
	auto x = data.map!(a => a[1 .. $]).array();
	return tuple(y, x);
}
</code></b>

### BLAS and Lapacke functions

Some funtions are imported from the `BLAS` and the `Lapacke` libraries. For more details of these algorithms see the [IBM ESSL documentation](https://www.ibm.com/support/knowledgecenter/en/SSFHY8/essl_content.html).

<b>
```
extern(C) @nogc nothrow{
void cblas_dgemm(in CBLAS_LAYOUT layout, in CBLAS_TRANSPOSE TransA,
                 in CBLAS_TRANSPOSE TransB, in int M, in int N,
                 in int K, in double alpha, in double  *A,
                 in int lda, in double  *B, in int ldb,
                 in double beta, double  *C, in int ldc);

alias int lapack_int;
alias int MKL_INT;
/* See IBM ESSL documentation for more details */
lapack_int LAPACKE_dgetrf(int matrix_layout, lapack_int m,
	        lapack_int n, double* a, lapack_int lda, 
	        lapack_int* ipiv);
lapack_int LAPACKE_dgetri(int matrix_layout, lapack_int n, 
	        double* a, lapack_int lda, in lapack_int* ipiv);
/* Function for calculating the norm of a vector */
double cblas_dnrm2 (in MKL_INT n , in double* x , in MKL_INT incx);
}
```
</b>

These functions are used in analysis and convenient interface functions are given below. The `norm2` $(||x||)$ interface function is given below:

<b>
```
/* Norm2 function */
double norm2(int incr = 1)(double[] x)
{
	return cblas_dnrm2(cast(int)x.length, x.ptr , incr);
}
```
</b>

Below is a function for the in-place inverse of a matrix

<b>```
/* Function for in place inverse of a matrix */
void inverse(int layout = LAPACK_ROW_MAJOR)(ref double[] matrix){
	int p = cast(int)sqrt(cast(double)matrix.length);
	int[] ipiv; ipiv.length = p;
	int info = LAPACKE_dgetrf(layout, p,
	            p, matrix.ptr, p, ipiv.ptr);
	assert(info == 0, "Illegal value from function LAPACKE_dgetrf");
	info = LAPACKE_dgetri(layout, p, 
	            matrix.ptr, p, ipiv.ptr);
	assert(info == 0, "Illegal value from function LAPACKE_dgetri");
}
```</b>

Below is a function for matrix multiplication

<b>
```
/* Matrix multiplication */
double[] matmult(CBLAS_LAYOUT layout = CblasRowMajor, CBLAS_TRANSPOSE transA = CblasNoTrans, 
	         CBLAS_TRANSPOSE transB = CblasNoTrans)
             (double[] a, double[] b, int[2] dimA, int[2] dimB){
	         
	         double alpha = 1.;
	         double beta = 0.;
	         int m = transA == CblasNoTrans ? dimA[0] : dimA[1];
	         int n = transB == CblasNoTrans ? dimB[1] : dimB[0];
	         double[] c; //set this length
	         
	         int k = transA == CblasNoTrans ? dimA[1] : dimA[0];
	         
	         int lda, ldb, ldc;
	         if(transA == CblasNoTrans)
	         	lda = layout == CblasRowMajor? k: m;
	         else
	         	lda = layout == CblasRowMajor? m: k;
	         
	         if(transB == CblasNoTrans)
	         	ldb = layout == CblasRowMajor? n: k;
	         else
	         	ldb = layout == CblasRowMajor? k: n;

	         ldc = layout == CblasRowMajor ? n : m;
	         c.length = layout == CblasRowMajor ? ldc*m : ldc*n;

	         cblas_dgemm(layout, transA, transB, m, n,
                 k, alpha, a.ptr, lda, b.ptr, ldb,
                 beta, c.ptr, ldc);
	         return c;
}
```
</b>

### D code for Gamma regression

The style of the code in the rest of this article happens to be written in a **functional style**. The functional style seems easier to write (if not read) these types of algorithms.

<b>```
/* Calculation of mu, k, xB, and n */
mixin template gParamCalcs()
{
	auto n = x.length, p = x[0].length;
	auto xB = zip(pars.repeat().take(n).array(), x)
	           .map!(a => zip(a[0], a[1])
	           	            .map!(a => a[0]*a[1])
	           	            .reduce!((a, b) => a + b))
	           .array();
	auto mu = xB.map!(a => exp(a)).array();
	auto k = (n - p)/zip(y, mu)
	                .map!(a => pow(a[0] - a[1], 2)/pow(a[1], 2))
	                .reduce!((a, b) => a + b);
}
```</b>

This is the implementation for the Gamma log-likelihood given previously omitting the Gamma function term which is not required

<b>```
/* This is a loglik for gamma distribution (omits gamma term) ... */
T gLogLik(T)(T[] pars, T[][] x, T[] y)
{
	mixin gParamCalcs;
	auto ll = zip(k.repeat().take(n).array(), xB, y, mu)
	          .map!(a => -a[0]*a[1] + a[0]*log(a[0]) + 
	          	          (a[0] - 1)*log(a[2]) - a[2]*a[0]/a[3])
	          .reduce!((a, b) => a + b);
	return ll;
}
```</b>

The functions for the Gamma gradient and curvature are given by:

<b>```
T[] gGradient(T)(T[] pars, T[][] x, T[] y)
{
	mixin gParamCalcs;
	auto output = zip(k.repeat().take(n).array(), x, y, mu)
	            .map!(a => 
	            	zip(a[0].repeat().take(a[1].length),
	            		a[1],
	            		a[2].repeat().take(a[1].length),
	            		a[3].repeat().take(a[1].length))
	            	.map!(a => -a[0]*a[1] + a[1]*a[0]*(a[2]/a[3]))
	            	.array())
	            .reduce!((a, b) => zip(a, b).map!(a => a[0] + a[1]).array())
	            .array();
	return output;
}
```
</b>

<b>```
auto gCurvature(T)(T[] pars, T[][] x, T[] y)
{
	mixin gParamCalcs;
	auto xPrime = zip(k.repeat().take(n).array(), x, y, mu)
	                .map!( a => 
	                	zip(a[0].repeat().take(a[1].length),
	            		    a[1],
	            		    a[2].repeat().take(a[1].length),
	            		    a[3].repeat().take(a[1].length))
	            		.map!(a => -a[2]*(a[0]/a[3])*a[1])
	            		.array())
	                .array();
    int[2] dims = [cast(int)n, cast(int)p];
	T[] curv = matmult!(CblasRowMajor, CblasTrans, CblasNoTrans)
             (xPrime.reduce!((a, b) => a ~ b).array(),
             	x.reduce!((a, b) => a ~ b).array(), 
             	dims, dims);
	return(curv);
}
```
</b>

The Newton-Raphson method are given by:

<b>```
T[] newtonRaphson(T, alias objFun = gLogLik, alias gradFun = gGradient, alias curveFun = gCurvature)
                     (T[] pars, T[][] x, T[] y, T tol = 1e-5, T ltol = 1e-5)
{
	T[] parsPrev = pars.dup;
	T ll = objFun(pars, x, y), llPrev = ll, llDiff = 1., parsDiff = 1.;
	int info, ipiv, n = cast(int)x.length, p = cast(int)x[0].length;
	T[] grad, curv, delta = T(0).repeat().take(p).array();
	while((parsDiff > tol) | (llDiff > ltol))
	{
		curv = curveFun(parsPrev, x, y);
		inverse(curv);
		grad = gradFun(parsPrev, x, y);
		// Multiply the gradient by the hessian
		delta = matmult!(CblasRowMajor, CblasNoTrans, CblasNoTrans)
                        (curv, grad, [p, p], [p, 1]);
		pars = zip(pars, delta)
		          .map!(a => a[0] - a[1])
		          .array();
		parsDiff = zip(pars, parsPrev)
		                .map!(a => pow((a[0] - a[1]), 2.))
		                .reduce!((a, b) => a + b)
		                .sqrt;
		ll = objFun(pars, x, y);
		llDiff = abs(ll - llPrev);
		parsPrev = pars;
		llPrev = ll;
	}
	return pars;
}
```
</b>

### D code for Stochastic Gradient Descent

The D implementation of the gradient function `nGradient` for the Normal log-likelihood is given below:

<b>```
/* For normal distribution gradient descent */
T[] nGradient(T)(T[] pars, T[] x, T y)
{
	T xSum = y - zip(pars, x).map!(a => a[0]*a[1])
	                   .reduce!((a, b) => a + b);
	T[] output = x.map!(a => a*xSum).array();
	return output;
}
```
</b>

The D implementation of the gradient descent function `gradientDescent` is given below:

<b>
```
T[] gradientDescent(T, alias gradFun = nGradient)
                     (T[] pars, T[][] x, T[] y, T alpha = 0.01,
                     	int nepochs = 500, T ptol = 1e-5)
{
	auto prevPars = pars.dup;
	auto temp = pars.dup;
	for(int i = 0; i < nepochs; ++i)
	{
		for(int j = 0; j < x.length; ++j)
		{
			prevPars = pars.dup;
			temp = gradFun(pars, x[j], y[j]);
			pars = zip(pars, temp)
			          .map!(a => a[0] + alpha*a[1])
			          .array();
			temp = zip(pars, prevPars)
			          .map!(a => a[0] - a[1])
			          .array();
			auto norm = norm2(temp);
			assert(norm != T.infinity, "infinite parameter issue");
			if(norm < ptol)
				break;
		}
	}
	return pars;
}

```
</b>

The use of the `newtonRaphson` function is given below:

<b>
```
auto z = read_yX("freeny.csv");
double[][] x;
double[] y;
y = z[0]; x = z[1];
writeln("Output: ", newtonRaphson([1.,1.,1.,1.,1.], x, y));
```
</b>

The output is given by:<br>
<b>`Output: [2.20867, 0.0125611, -0.0349353, 0.0378322, 0.0226276]`</b>

The use of the `gradientDescent` function is given below:

<b>```
z = read_yX("ex3.csv");
	y = z[0]; x = z[1];
	writeln("Output: ", gradientDescent([0.,0.,0.], x, y, 0.01, 500));
```</b>

The output is given by:<br><b>`Output: [200561, 503985, -30934]`</b>

The R scripts showing that these outputs are correct are given in the [Gradient Descent script](https://github.com/dataPulverizer/glm-stochastic-gradient-descent-d/blob/master/code/gradientDescent.r) and the [Newton-Raphson script](https://github.com/dataPulverizer/glm-stochastic-gradient-descent-d/blob/master/code/newtonRaphson.r)